In [1]:
import openai
from fastapi import FastAPI, Request
from fastapi.responses import Response
from twilio.twiml.messaging_response import MessagingResponse
from pyngrok import ngrok
import nest_asyncio
import uvicorn  

# Configurar OpenAI y Twilio
openai.api_key = 
TWILIO_ACCOUNT_SID = 
TWILIO_AUTH_TOKEN = 
TWILIO_WHATSAPP_NUMBER = 

# Inicializar FastAPI
app = FastAPI()

# Almacenar el estado de la conversación con cada usuario
usuarios_contexto = {}

# **Mensaje de bienvenida automático**
def mensaje_bienvenida():
    return (
        "👋 ¡Hola! Te has comunicado con el **Dr. Carlos García**.\n"
        "¿En qué puedo ayudarte hoy? Selecciona una opción:\n\n"
        "1️⃣ Agendar hora 📅\n"
        "2️⃣ Indicaciones preoperatorias 🏥\n"
        "3️⃣ Indicaciones postoperatorias 💊\n"
        "4️⃣ Otra consulta 📞"
    )

# Función para generar respuestas con GPT según el tipo de cirugía
def obtener_respuesta_gpt(tipo, tipo_cirugia):
    prompt = f"Soy un asistente médico. Necesito instrucciones claras y comprensibles sobre {tipo} para una cirugía de {tipo_cirugia}, deben ser cortas y precisas no mas de 800 caracteres. Responde de manera sencilla para que cualquier paciente pueda entender."

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": prompt}]
        )
        return response["choices"][0]["message"]["content"].strip()
    
    except Exception as e:
        print(f"⚠️ Error en GPT: {e}")
        return "Lo siento, no pude obtener las indicaciones en este momento."

# Función de respuesta del chatbot
def chatbot_response(user_id, message: str):
    try:
        message = message.lower().strip()
        print(f"📩 Mensaje recibido: {message}")

        # **Detectar si el usuario está iniciando la conversación**
        if user_id not in usuarios_contexto:
            usuarios_contexto[user_id] = "inicio"
            return mensaje_bienvenida()

        # Verificar si el usuario ya pidió pre o post operatorio y está eligiendo cirugía
        if usuarios_contexto[user_id] == "preoperatorio":
            usuarios_contexto.pop(user_id)
            return obtener_respuesta_gpt("preoperatorio", message)

        if usuarios_contexto[user_id] == "postoperatorio":
            usuarios_contexto.pop(user_id)
            return obtener_respuesta_gpt("postoperatorio", message)

        # Agendar hora: Mostrar ambas opciones en un solo mensaje
        if message in ["1", "agendar", "agendar hora"]:
            usuarios_contexto.pop(user_id, None)  # Eliminar cualquier estado previo
            return (
                "🏥 Puedes agendar tu hora en los siguientes lugares:\n\n"
                "✅ **Clínica MEDS**: Reserva tu hora aquí 👉 [🔗 Reserva de hora](https://www.meds.cl/reserva-horas-meds/)\n"
                "✅ **Hospital San Borja**: Llama al 📞 **800 600 700** para reservar tu cita."
            )

        # Preoperatorio: Preguntar tipo de cirugía
        if message in ["2", "preoperatorio", "indicaciones preoperatorias"]:
            usuarios_contexto[user_id] = "preoperatorio"
            return "¿Para qué tipo de cirugía necesitas indicaciones preoperatorias? (Ejemplo: cirugía general, cirugía ocular, cirugía ortopédica)"

        # Postoperatorio: Preguntar tipo de cirugía
        if message in ["3", "postoperatorio", "indicaciones postoperatorias"]:
            usuarios_contexto[user_id] = "postoperatorio"
            return "¿Para qué tipo de cirugía necesitas indicaciones postoperatorias? (Ejemplo: cirugía general, cirugía ocular, cirugía ortopédica)"

        # Otra consulta
        if message in ["4", "consulta", "otra consulta"]:
            return "🕒 Espera un momento, el Dr. Carlos García te contactará cuando pueda."

        # Si el mensaje no coincide con ninguna opción
        return "No entendí tu solicitud. Por favor, responde con el número de la opción que necesitas."

    except Exception as e:
        print(f"⚠️ Error en chatbot_response: {e}")
        return "Hubo un problema, intenta nuevamente más tarde."

# Endpoint de WhatsApp
@app.post("/whatsapp")
async def whatsapp_webhook(request: Request):
    form = await request.form()
    incoming_msg = form.get("Body").strip()
    user_id = form.get("From")  # ID único del usuario en WhatsApp
    
    print(f"📩 Mensaje recibido de WhatsApp ({user_id}): {incoming_msg}")
    
    response_text = chatbot_response(user_id, incoming_msg)
    
    resp = MessagingResponse()
    msg = resp.message(response_text)
    
    print(f"📤 Enviando respuesta a WhatsApp: {response_text}")
    
    return Response(str(resp), media_type="application/xml")  

# Configurar ngrok para obtener una URL pública
nest_asyncio.apply()
public_url = ngrok.connect(8000).public_url
print(f"🔥 URL pública para Twilio: {public_url}/whatsapp")

# Iniciar servidor FastAPI
uvicorn.run(app, host="0.0.0.0", port=8000)



INFO:     Started server process [9242]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔥 URL pública para Twilio: https://734b-191-125-46-122.ngrok-free.app/whatsapp
📩 Mensaje recibido de WhatsApp (whatsapp:+56995455248): Hola
📩 Mensaje recibido: hola
📤 Enviando respuesta a WhatsApp: 👋 ¡Hola! Te has comunicado con el **Dr. Carlos García**.
¿En qué puedo ayudarte hoy? Selecciona una opción:

1️⃣ Agendar hora 📅
2️⃣ Indicaciones preoperatorias 🏥
3️⃣ Indicaciones postoperatorias 💊
4️⃣ Otra consulta 📞
INFO:     98.81.239.57:0 - "POST /whatsapp HTTP/1.1" 200 OK
📩 Mensaje recibido de WhatsApp (whatsapp:+56995455248): Hola qué tal buenos días, necesito información porque necesito operarme
📩 Mensaje recibido: hola qué tal buenos días, necesito información porque necesito operarme
📤 Enviando respuesta a WhatsApp: No entendí tu solicitud. Por favor, responde con el número de la opción que necesitas.
INFO:     44.222.212.226:0 - "POST /whatsapp HTTP/1.1" 200 OK
📩 Mensaje recibido de WhatsApp (whatsapp:+56995455248): Hola
📩 Mensaje recibido: hola
📤 Enviando respuesta a WhatsApp: No e

t=2025-02-21T20:01:33-0300 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=5d7fc4c576ca clientid=f6eefa6ce930ac7e082f48a0d97ab77f
t=2025-02-21T20:01:33-0300 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=15833e8ef1f4 err="session closed"
t=2025-02-21T20:01:33-0300 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2025-02-21T20:01:33-0300 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2025-02-21T20:01:34-0300 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no 